In [1]:
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
import util
from sklearn.linear_model import LinearRegression
from timeit import timeit
import matplotlib.pyplot as plt

In [2]:
XFILE = 'data/imu_proc01-Feb-2018.csv'
YFILE = 'data/vicon_proc01-Feb-2018.csv'
X = np.genfromtxt(XFILE, delimiter=',')
X = X[:, 10:]  # wrist only
print('Input size:', X.shape)
y = np.genfromtxt(YFILE, delimiter=',')
print('Output size:', y.shape)

Input size: (4731, 10)
Output size: (4731, 5)


In [3]:
X_train, X_test, y_train, y_test = util.test_split(X, y, 1000, 0.1)
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
mlp = MLPRegressor(500, 'relu', alpha=0.1, max_iter=1000)
svr = SVR(C=10, gamma=0.1, epsilon=1, verbose=True)
svr = MultiOutputRegressor(svr, n_jobs=-1)
lr = LinearRegression()
models = [svr, mlp, lr]
names = ['SVM', 'MLP', 'LR']

In [7]:
train_time = np.zeros(len(models))
for i, model in enumerate(models):
    train_time[i] = timeit(lambda: model.fit(X_train, y_train), number=10) / 10

In [9]:
train_time

array([  2.97416838e+00,   2.26871364e+01,   1.95673765e-03])

In [9]:
for model in models:
    model.fit(X_train, y_train)

In [14]:
pred_time = np.zeros(len(models))
iters = 10
for i, model in enumerate(models):
    pred_time[i] = timeit(lambda: model.predict(X_test), number=iters) / iters

In [14]:
pred_time

array([  1.92861837e+00,   2.86696070e-03,   4.28641637e-05])

In [29]:
ind = np.arange(2)  # the x locations for the groups
plt.bar(ind, train_time[:2])
plt.ylabel('Training Time (s)')
plt.xticks(ind, ('SVM', 'MLP'))
plt.show()

In [30]:
ind = np.arange(2)  # the x locations for the groups
plt.bar(ind, pred_time[:2])
plt.ylabel('Prediction Time (s)')
plt.xticks(ind, ('SVM', 'MLP'))
plt.show()

In [37]:
N = 2
ind = np.arange(N)  # the x locations for the groups
width = 0.35       # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(ind, train_time[:2], width, color='black', log=True)
rects2 = ax.bar(ind + width, pred_time[:2], width, color='grey', log=True)
ax.set_ylabel('Time (s)')
ax.set_xticks(ind + width / 2)
ax.set_xticklabels(('SVM', 'MLP'))
ax.legend((rects1[0], rects2[0]), ('Training', 'Prediction'))
plt.savefig('results/model_time.png')

In [7]:
svr.get_params()

{'estimator': SVR(C=10, cache_size=200, coef0=0.0, degree=3, epsilon=1, gamma=0.1,
   kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False),
 'estimator__C': 10,
 'estimator__cache_size': 200,
 'estimator__coef0': 0.0,
 'estimator__degree': 3,
 'estimator__epsilon': 1,
 'estimator__gamma': 0.1,
 'estimator__kernel': 'rbf',
 'estimator__max_iter': -1,
 'estimator__shrinking': True,
 'estimator__tol': 0.001,
 'estimator__verbose': False,
 'n_jobs': -1}